<a href="https://colab.research.google.com/github/Maynul-haque/web_scraping_project/blob/main/hss_webscrapling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://103.247.238.92/webportal/pages/hss_menu_facility.php?facilitytype_id=29&division_id=&district_id='

r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')

In [ ]:
table_row = soup.find('div', {'class':"box-body"}).find_all('tr')


tab_link =[]
for i in table_row:
  if (i.find('td', text = 'Cumilla')) !=  None:
    tab_link.append(i)

name = [i.find('a').get_text() for i in tab_link ]


links_upazilla =['http://103.247.238.92/webportal/pages/'+i.find('a').get('href') for i in tab_link]

df = pd.DataFrame()
df['Name'] = name
df['Link'] = links_upazilla

In [ ]:

df.head(15)

,Name,Link
0,Daudkandi Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
1,Laksham Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
2,Debidwar Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
3,Titas Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
4,Muradnagar Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
5,Barura Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
6,Chaddagram Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
7,Comilla Sadar Daxin Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
8,Brahmanpara Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...
9,Meghna Upazila Health Complex,http://103.247.238.92/webportal/pages/hss_scor...


In [ ]:
upazilla = int(input('Enter serial number of Upazilla : \n --------------------------\n'))
print('You have Selected :',df['Name'][upazilla])
d = pd.read_html(df['Link'][upazilla])
table = int(input('Which table do you want to see? \n Select number of table \n------------------------------- \n0. Service Delivery- 6.4 , Per indicator mark- 1.28\n1. Health Workforce- 6.4 , Per indicator mark- 1.6\n2. Health Information System- 6.4 , Per indicator mark- 1.6\n3. Access to Essential Medicines/ Equipment, logistics/ Utilities/ Infrastructure- 6.37 , Per indicator mark- 0.91\n4 Leadership/ Governance/ Management- 6.43 , Indicator mark - 3.215\n5. Access- 12 ,  Indicator mark - 6\n6. Quality- 12 ,  Indicator mark - 3\n7. Coverage -12 ,  Indicator mark- 4\n8. Safety -12 , Indicator mark - 12\n9. Onsite Monitoring -20 , Indicator mark - 20 \n----------------------------------'))
d[table]

Enter serial number of Upazilla : 
 --------------------------
5
You have Selected : Barura Upazila Health Complex
Which table do you want to see? 
 Select number of table 
------------------------------- 
0. Service Delivery- 6.4 , Per indicator mark- 1.28
1. Health Workforce- 6.4 , Per indicator mark- 1.6
2. Health Information System- 6.4 , Per indicator mark- 1.6
3. Access to Essential Medicines/ Equipment, logistics/ Utilities/ Infrastructure- 6.37 , Per indicator mark- 0.91
4 Leadership/ Governance/ Management- 6.43 , Indicator mark - 3.215
5. Access- 12 ,  Indicator mark - 6
6. Quality- 12 ,  Indicator mark - 3
7. Coverage -12 ,  Indicator mark- 4
8. Safety -12 , Indicator mark - 12
9. Onsite Monitoring -20 , Indicator mark - 20 
----------------------------------1


,Indicators,Definition,Dataset,Period,Numerator,Denominator,Score Calculation,Score,Trend
0,Functionality of the biometric attendance mach...,Definition: No. of days attendance given by an...,Biometric Attendance,Monthly,NaN,NaN,Score = Result*Marks,0.0,NaN
1,Presence of average no. of staffs,Definition: No. of average employee present pe...,Biometric Attendance,Monthly,NaN,NaN,Score = Result*Marks,0.0,NaN
2,Completeness of HRM data (Facility),Definition: Timely completion of total 7 prede...,HRM,Monthly,7.0,7.0,Score = Result*Marks,1.6,NaN
3,Completeness of HRM data (Provider),Definition: Timely completion of total 10 pred...,HRM,Monthly,NaN,0.0,Score = Result*Marks,0.0,NaN
4,Total,NaN,NaN,NaN,NaN,NaN,NaN,1.6,NaN
